# Example 33: USGS Gauge Catalog Generation

This notebook demonstrates how to generate and use a standardized USGS gauge data catalog for your HEC-RAS project.

## Purpose

The gauge catalog generation function creates a standardized "USGS Gauge Data" folder (similar to the precipitation module's storm catalog) that:
- Discovers all active USGS gauges within project extent
- Downloads historical data for each gauge
- Creates master catalog for easy gauge discovery
- Provides standard location for engineering review and downstream functions

## Key Functions

- `generate_gauge_catalog()`: Create complete gauge catalog with metadata and data
- `load_gauge_catalog()`: Load gauge catalog from standard location
- `load_gauge_data()`: Load historical data for specific gauge
- `get_gauge_folder()`: Get path to gauge folder
- `update_gauge_catalog()`: Refresh catalog with latest data

## Example Project

We'll use the **Bald Eagle Creek** example project which has 2 active USGS gauges:
- USGS-01547200: Upstream gauge (265 sq mi drainage)
- USGS-01548005: Downstream gauge (562 sq mi drainage)

## Dependencies

Requires: `pip install dataretrieval geopandas tqdm`

## 1. Setup and Initialization

In [ ]:
# =============================================================================
# DEVELOPMENT MODE TOGGLE
# =============================================================================
USE_LOCAL_SOURCE = False  # <-- TOGGLE THIS

if USE_LOCAL_SOURCE:
    import sys
    from pathlib import Path
    local_path = str(Path.cwd().parent)
    if local_path not in sys.path:
        sys.path.insert(0, local_path)
    print(f"📁 LOCAL SOURCE MODE: Loading from {local_path}/ras_commander")
else:
    print("📦 PIP PACKAGE MODE: Loading installed ras-commander")

# Import ras-commander
from ras_commander import init_ras_project, ras, RasExamples
from ras_commander.usgs import (
    generate_gauge_catalog,
    load_gauge_catalog,
    load_gauge_data,
    get_gauge_folder,
    update_gauge_catalog
)

# Additional imports
import pandas as pd
import json

# Verify which version loaded
import ras_commander
print(f"✓ Loaded: {ras_commander.__file__}")

## 2. USGS API Key (Optional but Recommended)

### Rate Limits

USGS API has different rate limits depending on whether you have an API key:

| Tier | Rate Limit | Use Case |
|------|------------|----------|
| **Without API key** | ~5 requests/sec | Most users (default) |
| **With free API key** | ~10 requests/sec | Faster processing |

### Getting a Free API Key

Sign up for instant approval (no review process):

**https://api.waterdata.usgs.gov/signup/**

### Using an API Key

```python
from ras_commander.usgs import test_api_key

# Test your API key (replace with your actual key)
api_key = "YOUR_API_KEY_HERE"

if test_api_key(api_key):
    print("API key is valid!")
    # Use with generate_gauge_catalog
    summary = generate_gauge_catalog(
        api_key=api_key,
        rate_limit_rps=10.0  # Faster rate with API key
    )
else:
    print("API key is invalid - will use slower rate (5 req/sec)")
```

### Note

**This notebook runs without an API key** using the default 5 req/sec rate limit.  
If you have an API key, you can add it to the cells below for faster processing.

In [ ]:
# =============================================================================
# OPTIONAL: Test USGS API Key
# =============================================================================
# Uncomment and add your API key to test it before use

USE_API_KEY = False  # Set to True to use API key

if USE_API_KEY:
    from ras_commander.usgs import test_api_key
    
    # Replace with your actual API key from https://api.waterdata.usgs.gov/signup/
    api_key = "YOUR_API_KEY_HERE"
    
    print("Testing USGS API key...")
    if test_api_key(api_key):
        print("SUCCESS: API key is valid and functional!")
        print(f"You can now use rate_limit_rps=10.0 for faster processing")
    else:
        print("FAILED: API key validation failed")
        print("Will use default rate (5 req/sec) without API key")
        api_key = None
else:
    api_key = None
    print("Running without API key (5 req/sec rate limit)")
    print("Get a free API key at: https://api.waterdata.usgs.gov/signup/")

## 2. Extract Example Project

In [ ]:
# Extract Bald Eagle Creek project
project_path = RasExamples.extract_project("Balde Eagle Creek", output_path="example_projects_33_gauge_catalog_generation")

print(f"Project extracted to: {project_path}")

# Initialize project
init_ras_project(project_path, "6.6")

print(f"\nProject: {ras.project_name}")
print(f"Path: {ras.project_path}")
print(f"\nProject initialized successfully")

## 3. Generate Gauge Catalog

This will:
1. Find all USGS gauges within 50% buffer of project extent
2. Download 10 years of historical data (flow and stage)
3. Create standardized folder structure
4. Generate master catalog and documentation

In [ ]:
# Generate catalog with default settings
# Note: If USE_API_KEY=True above, this will use the validated API key
summary = generate_gauge_catalog(
    buffer_percent=50.0,         # Search within 50% buffer of project extent
    include_historical=True,     # Download historical data
    historical_years=10,         # Last 10 years of data
    parameters=['flow', 'stage'],  # Retrieve flow and stage data
    api_key=api_key,             # Use API key if provided (None otherwise)
    rate_limit_rps=10.0 if api_key else 5.0  # 10 req/sec with key, 5 without
)

# Display summary
print("
" + "="*60)
print("GAUGE CATALOG GENERATION SUMMARY")
print("="*60)
print(f"Gauges found: {summary['gauge_count']}")
print(f"Successfully processed: {summary['gauges_processed']}")
print(f"Failed: {summary['gauges_failed']}")
print(f"Output folder: {summary['output_folder']}")
print(f"Data size: {summary['data_size_mb']:.2f} MB")
print(f"Processing time: {summary['processing_time_sec']:.1f} seconds")
print("="*60)

## 4. Explore Catalog Structure

The catalog creates a standardized folder structure:

```
project_folder/
├── USGS Gauge Data/
│   ├── gauge_catalog.csv          # Master catalog
│   ├── gauge_locations.geojson    # Spatial data
│   ├── README.md                  # Documentation
│   ├── USGS-01547200/             # Individual gauge folders
│   │   ├── metadata.json
│   │   ├── historical_flow.csv
│   │   ├── historical_stage.csv
│   │   └── data_availability.json
│   └── USGS-01548005/
│       └── ...
```

In [ ]:
# List files in catalog folder
catalog_folder = Path(ras.project_path) / "USGS Gauge Data"

print("Catalog folder contents:")
print("\nTop-level files:")
for file in sorted(catalog_folder.glob('*')):
    if file.is_file():
        size_kb = file.stat().st_size / 1024
        print(f"  {file.name:30s} ({size_kb:8.1f} KB)")

print("\nGauge folders:")
for folder in sorted(catalog_folder.glob('USGS-*')):
    if folder.is_dir():
        file_count = len(list(folder.glob('*')))
        print(f"  {folder.name:30s} ({file_count} files)")

## 5. Load and Explore Master Catalog

In [ ]:
# Load catalog using helper function
catalog = load_gauge_catalog()

print(f"Loaded catalog with {len(catalog)} gauges\n")

# Display key information
print("Gauge Catalog:")
print("-" * 120)
display_cols = ['site_id', 'station_name', 'drainage_area_sqmi', 'upstream_downstream', 
                'distance_to_project_km', 'parameters_available']
print(catalog[display_cols].to_string(index=False))
print("-" * 120)

## 6. Load Gauge Metadata

In [ ]:
# Load metadata for first gauge
site_id = catalog.iloc[0]['site_id']
gauge_folder = get_gauge_folder(site_id)
metadata_file = gauge_folder / "metadata.json"

with open(metadata_file, 'r') as f:
    metadata = json.load(f)

print(f"Metadata for USGS-{site_id}:")
print("="*60)
print(f"Station: {metadata['station_name']}")
print(f"Location: {metadata['location']['latitude']:.4f}, {metadata['location']['longitude']:.4f}")
print(f"State: {metadata['location']['state']}")
print(f"County: {metadata['location']['county']}")
print(f"Drainage Area: {metadata['drainage_area_sqmi']} sq mi")
print(f"Gage Datum: {metadata['gage_datum_ft']} ft")
print(f"Active: {metadata['active']}")
print(f"\nAvailable Parameters: {', '.join(metadata['available_parameters'])}")
print(f"\nPeriod of Record:")
print(f"  Start: {metadata['period_of_record']['start']}")
print(f"  End: {metadata['period_of_record']['end']}")
print(f"  Years: {metadata['period_of_record']['years']}")
print(f"\nLast Updated: {metadata['last_updated']}")
print("="*60)

## 7. Load Data Availability Information

In [ ]:
# Load data availability for first gauge
availability_file = gauge_folder / "data_availability.json"

with open(availability_file, 'r') as f:
    availability = json.load(f)

print(f"Data Availability for USGS-{site_id}:")
print("="*60)

for param, info in availability.items():
    print(f"\n{param.upper()}:")
    print(f"  Available: {info['available']}")
    if info['available']:
        print(f"  Date Range: {info['start_date']} to {info['end_date']}")
        print(f"  Record Count: {info['record_count']:,}")
        print(f"  Completeness: {info['completeness']*100:.1f}%")
        if info['gaps']:
            print(f"  Gaps Found: {len(info['gaps'])}")
            for gap in info['gaps'][:3]:  # Show first 3 gaps
                print(f"    - {gap['start']} to {gap['end']} ({gap['days']} days)")

## 8. Load Historical Data Using Helper Function

In [ ]:
# Load flow data for first gauge
flow_data = load_gauge_data(site_id, parameter='flow')

print(f"Flow Data for USGS-{site_id}:")
print("="*60)
print(f"Records: {len(flow_data):,}")
print(f"Date Range: {flow_data['datetime'].min()} to {flow_data['datetime'].max()}")
print(f"\nFlow Statistics:")
print(f"  Mean: {flow_data['value'].mean():.1f} cfs")
print(f"  Median: {flow_data['value'].median():.1f} cfs")
print(f"  Min: {flow_data['value'].min():.1f} cfs")
print(f"  Max: {flow_data['value'].max():.1f} cfs")
print(f"\nFirst 5 records:")
print(flow_data.head())
print("="*60)

In [ ]:
# Load stage data
stage_data = load_gauge_data(site_id, parameter='stage')

print(f"Stage Data for USGS-{site_id}:")
print("="*60)
print(f"Records: {len(stage_data):,}")
print(f"Date Range: {stage_data['datetime'].min()} to {stage_data['datetime'].max()}")
print(f"\nStage Statistics:")
print(f"  Mean: {stage_data['value'].mean():.2f} ft")
print(f"  Median: {stage_data['value'].median():.2f} ft")
print(f"  Min: {stage_data['value'].min():.2f} ft")
print(f"  Max: {stage_data['value'].max():.2f} ft")
print(f"\nFirst 5 records:")
print(stage_data.head())
print("="*60)

## 9. Plot Historical Data

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Create figure with 2 subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Plot flow
ax1.plot(flow_data['datetime'], flow_data['value'], 'b-', linewidth=0.8, alpha=0.7)
ax1.set_ylabel('Flow (cfs)', fontsize=12, fontweight='bold')
ax1.set_title(f"USGS-{site_id}: {metadata['station_name']}", fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.set_ylim(bottom=0)

# Plot stage
ax2.plot(stage_data['datetime'], stage_data['value'], 'g-', linewidth=0.8, alpha=0.7)
ax2.set_ylabel('Stage (ft)', fontsize=12, fontweight='bold')
ax2.set_xlabel('Date', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(bottom=0)

# Format x-axis
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax2.xaxis.set_major_locator(mdates.YearLocator())
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print(f"\nPlot shows 10 years of flow and stage data for USGS-{site_id}")

## 10. Process All Gauges in Catalog

In [ ]:
# Summary statistics for all gauges
print("Summary for all gauges:")
print("="*80)

for idx, gauge in catalog.iterrows():
    site_id = gauge['site_id']
    name = gauge['station_name']
    drainage = gauge['drainage_area_sqmi']
    position = gauge['upstream_downstream']
    distance = gauge['distance_to_project_km']
    
    print(f"\nUSGS-{site_id}: {name}")
    print(f"  Position: {position.title()} ({distance:.1f} km from project)")
    print(f"  Drainage: {drainage} sq mi")
    
    # Check if flow data file exists before loading
    gauge_folder = get_gauge_folder(site_id)
    flow_file = gauge_folder / "historical_flow.csv"
    
    if flow_file.exists():
        flow = load_gauge_data(site_id, parameter='flow')
        print(f"  Flow: {len(flow):,} records, mean={flow['value'].mean():.0f} cfs, max={flow['value'].max():.0f} cfs")
    else:
        print(f"  Flow: No data available")
    
    # Check if stage data file exists before loading
    stage_file = gauge_folder / "historical_stage.csv"
    
    if stage_file.exists():
        stage = load_gauge_data(site_id, parameter='stage')
        print(f"  Stage: {len(stage):,} records, mean={stage['value'].mean():.2f} ft, max={stage['value'].max():.2f} ft")
    else:
        print(f"  Stage: No data available")

print("\n" + "="*80)

## 11. Use Catalog Data for Boundary Conditions

The catalog data can be easily used with other USGS functions:

In [ ]:
from ras_commander.usgs import generate_flow_hydrograph_table

# Select upstream gauge for BC generation
upstream_gauge = catalog[catalog['upstream_downstream'] == 'upstream'].iloc[0]
site_id = upstream_gauge['site_id']

print(f"Using USGS-{site_id} for boundary condition:")
print(f"  Station: {upstream_gauge['station_name']}")
print(f"  Drainage: {upstream_gauge['drainage_area_sqmi']} sq mi\n")

# Load flow data
flow = load_gauge_data(site_id, parameter='flow')

# Get last 7 days (168 hours)
recent_flow = flow.tail(168).copy()

print(f"Using last {len(recent_flow)} hourly values for BC")
print(f"Date range: {recent_flow['datetime'].min()} to {recent_flow['datetime'].max()}")
print(f"Flow range: {recent_flow['value'].min():.0f} to {recent_flow['value'].max():.0f} cfs\n")

# Generate HEC-RAS format hydrograph table
bc_table = generate_flow_hydrograph_table(
    flow_values=recent_flow['value'],
    interval='1HOUR'
)

print("Generated boundary condition table:")
print(bc_table[:500])  # Show first 500 characters
print(f"\n... ({len(bc_table)} characters total)")

## 12. Update Catalog (Add New Data)

The `update_gauge_catalog()` function refreshes existing gauges with new data:

In [ ]:
# Update catalog with latest data (last 30 days)
update_summary = update_gauge_catalog()

print("\nCatalog Update Summary:")
print("="*60)
print(f"Gauges updated: {update_summary['gauges_updated']}")
print(f"Gauges failed: {update_summary['gauges_failed']}")
print(f"Processing time: {update_summary['processing_time_sec']:.1f} seconds")
print("="*60)

## 13. Custom Catalog Configuration

You can customize the catalog generation:

In [ ]:
# Example: Generate catalog with custom settings
# (Don't run this cell if you want to keep existing catalog)

if False:  # Set to True to run
    custom_summary = generate_gauge_catalog(
        buffer_percent=100.0,        # Wider search area (2x project extent)
        include_historical=True,
        historical_years=20,         # More historical data
        parameters=['flow', 'stage', 'temperature'],  # Additional parameters
        output_folder=None           # Use default location
    )
    
    print("Custom catalog generated:")
    print(f"  Gauges: {custom_summary['gauge_count']}")
    print(f"  Data size: {custom_summary['data_size_mb']:.2f} MB")

## Summary

This notebook demonstrated:

1. ✅ **Catalog Generation**: One-command gauge discovery and data download
2. ✅ **Standard Structure**: Consistent folder organization across projects
3. ✅ **Metadata Management**: Complete gauge information in JSON format
4. ✅ **Data Loading**: Easy access to historical gauge data
5. ✅ **Integration**: Seamless use with boundary condition generation
6. ✅ **Updates**: Refresh catalog with latest data

## Key Takeaways

- **Standard Location**: `project_folder/USGS Gauge Data/` (like precipitation module)
- **One Command**: `generate_gauge_catalog()` does everything
- **Engineering Review**: Master catalog CSV for easy gauge assessment
- **Downstream Functions**: Standard location for automated workflows
- **Documentation**: Auto-generated README with usage instructions

## Related Examples

- Example 29: USGS Gauge Data Integration (basic retrieval)
- Example 30: Real-Time Monitoring (live gauge data)
- Example 31: BC Generation from Live Gauge (boundary conditions)
- Example 32: Model Validation with USGS (calibration metrics)

## Next Steps

With the catalog generated, you can:
- Review all available gauges in one location
- Use gauge data for boundary condition generation
- Perform model validation with observed data
- Include catalog in project deliverables for documentation